In [1]:
import os
import tensorflow as tf
from tfx import v1 as tfx

2023-10-30 14:40:46.023631: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 14:40:46.577734: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 14:40:46.581083: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 14:40:48.667770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


# Set up variables

In [2]:
SCHEMA_PIPELINE_NAME = 'doc_to_sleep_schema'
PIPELINE_NAME = 'doc_to_sleep'

# Output directory to store artifacts generated from the pipelines.
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite database file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME, 'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO) # Set default logging level.

In [23]:
# DATA_ROOT = os.path.join('converted_data', 'poly3.csv')
DATA_ROOT = 'converted_data'

# Generate a preliminary schema

In [24]:
from tfx.orchestration.metadata import Metadata, sqlite_metadata_connection_config
from tfx.v1.proto import Input
import tfx.proto.example_gen_pb2 as pb2

def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
    '''Creates a pipeline for schema generation'''
    # Brings data into the pipeline.
    input = Input(splits=[
                pb2.Input.Split(name='test', pattern='test/*'),
                # pb2.Input.Split(name='train', pattern='train/*'),
                # pb2.Input.Split(name='eval', pattern='eval/*')
            ])
    
    # output = pb2.Output(
    # split_config=pb2.SplitConfig(splits=[
    #     pb2.SplitConfig.Split(name='train', hash_buckets=1),
    #     pb2.SplitConfig.Split(name='eval', hash_buckets=1),
    #     pb2.SplitConfig.Split(name='test', hash_buckets=1)
    # ]))
    
    example_gen = tfx.components.CsvExampleGen(input_base=data_root, input_config=input)

    # Computes statistics over data for visualization and schema generation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples']
    )

    # Generates schema based on the generated statistics.
    schema_gen = tfx.components.SchemaGen(
        statistics=statistics_gen.outputs['statistics'],
        infer_feature_shape=True
    )

    components = [example_gen,
                  statistics_gen,
                  schema_gen]
    
    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=sqlite_metadata_connection_config(metadata_path),
        components=components)

Run the pipeline.

In [25]:
tfx.orchestration.LocalDagRunner().run(
    _create_schema_pipeline(
        pipeline_name=SCHEMA_PIPELINE_NAME,
        pipeline_root=SCHEMA_PIPELINE_ROOT,
        data_root=DATA_ROOT,
        metadata_path=SCHEMA_METADATA_PATH
    )
)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "SchemaGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.schema_gen.executor.Executor"
    }
  }
}
executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_gen.driver.FileBasedDriver"
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metada

# Review outputs of the schema pipeline

In [26]:
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.portable.mlmd import execution_lib

def get_latest_artifacts(metadata, pipeline_name, component_id):
    '''Output artifacts of the latest run of the component.'''
    context = metadata.store.get_context_by_type_and_name(
        'node', f'{pipeline_name}.{component_id}'
    )
    executions = metadata.store.get_executions_by_context(context.id)
    latest_execution = max(executions,
    key=lambda e:e.last_update_time_since_epoch)
    return execution_lib.get_output_artifacts(metadata, latest_execution.id)

from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
    '''Visualizes artifacts using standard visualization modules.'''
    for artifact in artifacts:
        visualization = visualizations.get_registry().get_visualization(
            artifact.type_name
        )
        if visualization:
            visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations

standard_visualizations.register_standard_visualizations()

Examine the outputs from the pipeline execution.

In [27]:
from tfx.orchestration.metadata import Metadata, sqlite_metadata_connection_config
from tfx.types import standard_component_specs

metadata_connection_config = sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH
)

with Metadata(metadata_connection_config) as metadata_handler:
    # Find output artifacts from MLMD.
    stat_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME,
                                           'StatisticsGen')
    stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

    schema_gen_output = get_latest_artifacts(metadata_handler,
                                             SCHEMA_PIPELINE_NAME,
                                             'SchemaGen')
    schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

INFO:absl:MetadataStore with DB connection initialized


Output from StatisticsGen

In [28]:
visualize_artifacts(stats_artifacts)

Output from SchemaGen

In [29]:
visualize_artifacts(schema_artifacts)

,Type,Presence,Valency,Domain
Feature name,,,,
'BR flow',FLOAT,required,,-
'ECG *',FLOAT,required,,-
'EMG right',FLOAT,required,,-
'HR central',FLOAT,required,,-
'Light',FLOAT,required,,-
'Obstr',FLOAT,required,,-
'Position',FLOAT,required,,-
'Pulse',FLOAT,required,,-
'Pulsewave',FLOAT,required,,-


Export the schema for future use

In [30]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

'schema/schema.pbtxt'

Run the cell below to view the exported schema.

In [31]:
!cat {SCHEMA_PATH}/*

feature {
  name: "BR flow"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "ECG *"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "EMG right"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "HR central"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Light"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Obstr"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Position"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    

# Write model training code

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

# Insert the target label below
_LABEL_KEY = None

_TRAIN_BATCH_SIZE = None
_EVAL_BATCH_SIZE = None

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int=200) -> tf.data.Dataset:
    '''Generates features and label for training.
    
    Args:
        file_pattern: List of paths or patterns of input tfrecord files.
        data_accessor: DataAccessor for converting input to RecordBatch.
        schema: schema of the input data.
        batch_size: representing the number of consecutive elements of
            returned dataset to combine in a single batch.
            
    Returns:
        A dataset that contains (features, indices) tuple where features is a
            dictionary of Tensors, and indices is a single Tensor of label indices.
    '''
    return data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(
            batch_size=batch_size, label_key=_LABEL_KEY
        ),
        schema=schema
    ).repeat()

def _build_keras_model(schema: schema_pb2.Schema) -> tf.keras.Model:
    '''Creates a DNN Keras model for the data.
    
    Returns:
        A Keras model.
    '''
    # Extract features from the schema except for the target label.
    feature_keys = [f.name for f in schema.feature if f.name != _LABEL_KEY]
    inputs = [keras.layers.Input(shape=(1,), name=f) for f in feature_keys]

    # Build and compile model below

# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
    '''Train the model based on given args.
    
    Args:
        fn_args: Holds args used to train the model as name/value pairs.
    '''
    # Reads in schema file passed to the Trainer component.
    schema = tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema_pbs.Schema())

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        schema,
        batch_size=_TRAIN_BATCH_SIZE
    )
    eval_dataset = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        schema,
        batch_size=_EVAL_BATCH_SIZE
    )

    model = _build_keras_model(schema)
    model.fit(train_dataset,
              steps_per_epoch=fn_args.train_steps,
              validation_data=eval_dataset,
              validation_steps=fn_args.eval_steps)
    
    # The result of the training should be saved in 'fn_args.serving_model_dir' directory
    model.save(fn_args.serving_model_dir, save_format='tf')

Write a pipeline definition.

In [ ]:
def _create_pipeline(pipeline_name: str,
                     pipeline_root: str,
                     data_root: str,
                     schema_path: str,
                     module_file: str,
                     serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
    '''Creates a pipeline using predefined schema with TFX.'''
    # Brings data into the pipeline.
    example_gen = tfx.components.CsvExampleGen(input_base=data_root)

    # Computes statistics over data for visualization and example validation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples']
    )

    # Import the schema.
    schema_importer = tfx.dsl.Importer(
        source_uri=schema_path,
        artifact_type=tfx.types.standard_artifacts.Schema
    ).with_id('schema_importer')

    # Performs anomaly detection based on statistics and data schema.
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_importer.outputs['result']
    )

    # Uses user-provided Python function that trains a model.
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_importer.outputs['result'],
        train_args=tfx.proto.TrainArgs(num_steps=100),
        eval_args=tfx.proto.EvalArgs(num_steps=5)
    )

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        push_destination=tfx.proto.PushDestination(
            filesystem=tfx.proto.PushDestination.Filesystem(
                base_directory=serving_model_dir
            )
        )
    )

    components = [example_gen,
                  statistics_gen,
                  schema_importer,
                  example_validator,
                  trainer,
                  pusher]
    
    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=sqlite_metadata_connection_config(metadata_path),
        components=components
    )

The cell beneath runs the pipeline.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        schema_path=SCHEMA_PATH,
        module_file=_trainer_module_file,
        serving_model_dir=SERVING_MODEL_DIR,
        metadata_path=METADATA_PATH
    )
)

Run the cell below to examine the outputs of the pipeline.

In [ ]:
metadata_connection_config = sqlite_metadata_connection_config(
    METADATA_PATH
)

with Metadata(metadata_connection_config) as metadata_handler:
    ev_output=get_latest_artifacts(metadata_handler,
                                   PIPELINE_NAME,
                                   'ExampleValidator')
    anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

Run the cell below to visualize the anomalies from ExampleValidator.

In [ ]:
visualize_artifacts(anomalies_artifacts)